In [24]:
import pandas as pd
import os
from ollama import chat

#### Load dataset

In [10]:
data_set = pd.read_csv("../scripts/data/tmp/phenopacket_dataset.csv")
data_set.head()

,pmid,input,truth
0,PMID_27656288,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
1,PMID_28503313,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
2,PMID_25469541,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
3,PMID_20932317,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...
4,PMID_20932317,/Users/rouvenreuter/Documents/Projects/P5/scri...,/Users/rouvenreuter/Documents/Projects/P5/scri...


In [11]:
data_set = data_set.drop_duplicates("pmid")

In [19]:
# For now we load the generated case summaries.
input_data = list()
for file_path in data_set['input']:
    try:
        with open(file_path, "r") as f:
            data = f.read()
            input_data.append(data.split("[text]")[-1]) 
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
input_data[0]

'\nThe patient is the fifth child of healthy, non-consanguineous parents. After an uneventful pregnancy, he was born at 35 weeks gestation following the premature rupture of the fetal membranes. His birth weight was 1,924 g (−1.3 SD). Because of an unknown cause, perforation of the terminal ileum occurred at the age of 2 days, and an ileostomy was performed.\nAfter follow-up with central venous nutrition and tube feeding, closure of the ileostomy was performed at the age of 52 days, and he left the hospital at the age of 73 days. He achieved head control at the age of 9 months and walked independently at the age of 18 months. At the age of 1 year, the patient was diagnosed with hypotonia and kyphoscoliosis, and treated with a brace at another clinic.\nHis kyphoscoliosis was non-responsive to the brace and increased in severity over time. At the age of 9 years, the patient was suspected to have a connective tissue disorder because he showed muscle weakness and joint hypermobility. There

#### Inference

In [21]:
prompt = "Please create a valid Phenopacket from the following text:"
model = "deepseek-r1:8b" 

In [ ]:

for text in input_data:
    response = chat(
        model=model,
        messages=[{"role": "user", "content": f"{prompt} {text} [EOS]"}],
    )
    break


In [ ]:
try:
    phenopacket_json = json.loads(response["message"]["content"])
    with open(f"{out_dir}/{file_name.split(".")[0]}.json", "w") as f:
        json.dump(phenopacket_json, f)
except json.decoder.JSONDecodeError:
    click.secho(
        message=f"{model} did not convert {file_name} into valid json format",
        err=True,
        fg="red",
    )